# K线形态识别

## 📋 概述

K线形态识别是技术分析的核心技能，通过识别不同的K线形态，可以判断市场趋势的转折点和延续信号。本章节将详细介绍各种K线形态的特征、市场含义和识别方法。

**学习方式**：本文件是Jupyter Notebook格式，你可以边看边运行代码，通过实际图表识别K线形态，加深理解。


## 📦 环境准备

首先安装必要的Python库：


In [ ]:
# 安装必要的库（如果还没有安装）
# !pip install pandas numpy matplotlib mplfinance akshare

# 导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import akshare as ak
from datetime import datetime, timedelta

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完成！")


## 📖 核心内容

### 1. 单根K线形态

单根K线形态是最基础的K线形态，通过分析单根K线的实体和影线，可以判断多空力量的对比。

#### 1.1 大阳线和大阴线

**大阳线**：
- 实体大，几乎没有影线
- 表示强烈的上涨信号
- 买方力量强

**大阴线**：
- 实体大，几乎没有影线
- 表示强烈的下跌信号
- 卖方力量强

让我们通过代码生成和识别这些形态：


In [ ]:
# 创建示例数据，展示不同的K线形态
def create_candle_pattern(pattern_name, base_price=10):
    """创建特定K线形态的示例数据"""
    patterns = {
        '大阳线': {
            'open': base_price,
            'high': base_price * 1.08,
            'low': base_price * 0.99,
            'close': base_price * 1.07
        },
        '大阴线': {
            'open': base_price,
            'high': base_price * 1.01,
            'low': base_price * 0.92,
            'close': base_price * 0.93
        },
        '十字星': {
            'open': base_price,
            'high': base_price * 1.02,
            'low': base_price * 0.98,
            'close': base_price
        },
        'T字线': {
            'open': base_price,
            'high': base_price * 1.01,
            'low': base_price * 0.95,
            'close': base_price
        },
        '倒T字线': {
            'open': base_price,
            'high': base_price * 1.05,
            'low': base_price * 0.99,
            'close': base_price
        },
        '锤子线': {
            'open': base_price * 0.98,
            'high': base_price * 1.01,
            'low': base_price * 0.92,
            'close': base_price * 0.99
        },
        '倒锤子线': {
            'open': base_price * 1.02,
            'high': base_price * 1.08,
            'low': base_price * 0.99,
            'close': base_price * 1.01
        }
    }
    return patterns.get(pattern_name, patterns['大阳线'])

# 创建单根K线形态示例
single_patterns = ['大阳线', '大阴线', '十字星', 'T字线', '倒T字线', '锤子线', '倒锤子线']
dates = pd.date_range('2024-01-01', periods=len(single_patterns), freq='D')

single_data = []
for i, pattern in enumerate(single_patterns):
    p = create_candle_pattern(pattern, base_price=10 + i * 0.5)
    single_data.append({
        'Open': p['open'],
        'High': p['high'],
        'Low': p['low'],
        'Close': p['close'],
        'Volume': 1000000
    })

single_df = pd.DataFrame(single_data, index=dates)

# 绘制单根K线形态
fig, ax = plt.subplots(figsize=(15, 8))
mpf.plot(single_df, type='candle', style='charles', ax=ax, volume=False)

# 添加标签
for i, pattern in enumerate(single_patterns):
    ax.text(i, single_df.iloc[i]['High'] * 1.02, pattern, 
            ha='center', fontsize=10, fontweight='bold')

ax.set_title('单根K线形态示例', fontsize=16, fontweight='bold')
ax.set_xlabel('日期')
ax.set_ylabel('价格')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("单根K线形态展示完成！")


In [ ]:
# 创建双根K线组合示例
def create_two_candle_pattern(pattern_name, base_price=10):
    """创建双根K线组合形态"""
    patterns = {
        '看涨吞没': [
            {'open': base_price, 'high': base_price * 1.01, 'low': base_price * 0.97, 'close': base_price * 0.98},
            {'open': base_price * 0.97, 'high': base_price * 1.05, 'low': base_price * 0.96, 'close': base_price * 1.04}
        ],
        '看跌吞没': [
            {'open': base_price, 'high': base_price * 1.02, 'low': base_price * 0.99, 'close': base_price * 1.01},
            {'open': base_price * 1.02, 'high': base_price * 1.03, 'low': base_price * 0.94, 'close': base_price * 0.95}
        ],
        '刺透形态': [
            {'open': base_price, 'high': base_price * 1.01, 'low': base_price * 0.92, 'close': base_price * 0.93},
            {'open': base_price * 0.92, 'high': base_price * 1.02, 'low': base_price * 0.91, 'close': base_price * 0.98}
        ],
        '乌云盖顶': [
            {'open': base_price, 'high': base_price * 1.05, 'low': base_price * 0.99, 'close': base_price * 1.04},
            {'open': base_price * 1.05, 'high': base_price * 1.06, 'low': base_price * 0.97, 'close': base_price * 0.98}
        ]
    }
    return patterns.get(pattern_name, patterns['看涨吞没'])

# 创建双根K线组合数据
two_patterns = ['看涨吞没', '看跌吞没', '刺透形态', '乌云盖顶']
two_dates_list = []
two_data_list = []

for i, pattern in enumerate(two_patterns):
    dates = pd.date_range(f'2024-01-0{i+1}', periods=2, freq='D')
    pattern_data = create_two_candle_pattern(pattern, base_price=10 + i * 2)
    
    data = []
    for p in pattern_data:
        data.append({
            'Open': p['open'],
            'High': p['high'],
            'Low': p['low'],
            'Close': p['close'],
            'Volume': 1000000
        })
    
    df = pd.DataFrame(data, index=dates)
    two_dates_list.append(dates)
    two_data_list.append(df)

# 绘制双根K线组合
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, (pattern, df) in enumerate(zip(two_patterns, two_data_list)):
    mpf.plot(df, type='candle', style='charles', ax=axes[idx], volume=False)
    axes[idx].set_title(f'{pattern}', fontsize=14, fontweight='bold')
    axes[idx].set_ylabel('价格')

plt.suptitle('双根K线组合形态示例', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("双根K线组合形态展示完成！")


### 3. 三根K线组合

三根K线组合是最重要的反转形态，包括早晨之星和黄昏之星。

#### 3.1 早晨之星（看涨反转）

**形态特征**：
- 第一根：大阴线（下跌趋势）
- 第二根：小实体（十字星或小K线），形成跳空
- 第三根：大阳线，收盘价超过第一根中点

**市场含义**：
- 下跌趋势可能结束
- 可能见底反转
- 是强烈的买入信号

#### 3.2 黄昏之星（看跌反转）

**形态特征**：
- 第一根：大阳线（上涨趋势）
- 第二根：小实体（十字星或小K线），形成跳空
- 第三根：大阴线，收盘价低于第一根中点

**市场含义**：
- 上涨趋势可能结束
- 可能见顶反转
- 是强烈的卖出信号

让我们通过代码展示这些重要形态：


In [ ]:
# 创建三根K线组合示例
def create_three_candle_pattern(pattern_name, base_price=10):
    """创建三根K线组合形态"""
    patterns = {
        '早晨之星': [
            {'open': base_price, 'high': base_price * 1.01, 'low': base_price * 0.92, 'close': base_price * 0.93},
            {'open': base_price * 0.90, 'high': base_price * 0.92, 'low': base_price * 0.88, 'close': base_price * 0.91},
            {'open': base_price * 0.91, 'high': base_price * 1.02, 'low': base_price * 0.90, 'close': base_price * 1.00}
        ],
        '黄昏之星': [
            {'open': base_price, 'high': base_price * 1.05, 'low': base_price * 0.99, 'close': base_price * 1.04},
            {'open': base_price * 1.06, 'high': base_price * 1.08, 'low': base_price * 1.04, 'close': base_price * 1.05},
            {'open': base_price * 1.05, 'high': base_price * 1.06, 'low': base_price * 0.97, 'close': base_price * 0.98}
        ],
        '红三兵': [
            {'open': base_price, 'high': base_price * 1.02, 'low': base_price * 0.99, 'close': base_price * 1.01},
            {'open': base_price * 1.01, 'high': base_price * 1.03, 'low': base_price * 1.00, 'close': base_price * 1.02},
            {'open': base_price * 1.02, 'high': base_price * 1.04, 'low': base_price * 1.01, 'close': base_price * 1.03}
        ],
        '三只乌鸦': [
            {'open': base_price, 'high': base_price * 1.01, 'low': base_price * 0.98, 'close': base_price * 0.99},
            {'open': base_price * 0.99, 'high': base_price * 1.00, 'low': base_price * 0.97, 'close': base_price * 0.98},
            {'open': base_price * 0.98, 'high': base_price * 0.99, 'low': base_price * 0.96, 'close': base_price * 0.97}
        ]
    }
    return patterns.get(pattern_name, patterns['早晨之星'])

# 创建三根K线组合数据
three_patterns = ['早晨之星', '黄昏之星', '红三兵', '三只乌鸦']
three_dates_list = []
three_data_list = []

for i, pattern in enumerate(three_patterns):
    dates = pd.date_range(f'2024-01-0{i+1}', periods=3, freq='D')
    pattern_data = create_three_candle_pattern(pattern, base_price=10 + i * 2)
    
    data = []
    for p in pattern_data:
        data.append({
            'Open': p['open'],
            'High': p['high'],
            'Low': p['low'],
            'Close': p['close'],
            'Volume': 1000000
        })
    
    df = pd.DataFrame(data, index=dates)
    three_dates_list.append(dates)
    three_data_list.append(df)

# 绘制三根K线组合
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, (pattern, df) in enumerate(zip(three_patterns, three_data_list)):
    mpf.plot(df, type='candle', style='charles', ax=axes[idx], volume=False)
    axes[idx].set_title(f'{pattern}', fontsize=14, fontweight='bold')
    axes[idx].set_ylabel('价格')

plt.suptitle('三根K线组合形态示例', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("三根K线组合形态展示完成！")


### 4. 实际股票数据中的K线形态识别

现在让我们从实际股票数据中识别K线形态：


In [ ]:
# 获取实际股票数据
try:
    # 获取股票数据（以平安银行000001为例）
    stock_data = ak.stock_zh_a_hist(symbol="000001", period="daily", 
                                     start_date="20240101", end_date="20241231", 
                                     adjust="qfq")
    
    # 重命名列名
    stock_data.columns = ['日期', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率']
    stock_data['日期'] = pd.to_datetime(stock_data['日期'])
    stock_data = stock_data.set_index('日期')
    stock_data = stock_data[['开盘', '最高', '最低', '收盘', '成交量']]
    stock_data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    
    print("数据获取成功！")
    print(f"数据形状：{stock_data.shape}")
    
except Exception as e:
    print(f"数据获取失败：{e}")
    print("使用模拟数据演示...")
    # 创建模拟数据
    dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')
    dates = dates[dates.weekday < 5]  # 只保留工作日
    
    np.random.seed(42)
    prices = 10 + np.cumsum(np.random.randn(len(dates)) * 0.1)
    
    stock_data = pd.DataFrame({
        'Open': prices + np.random.randn(len(dates)) * 0.05,
        'High': prices + np.abs(np.random.randn(len(dates)) * 0.1),
        'Low': prices - np.abs(np.random.randn(len(dates)) * 0.1),
        'Close': prices + np.random.randn(len(dates)) * 0.05,
        'Volume': np.random.randint(1000000, 10000000, len(dates))
    }, index=dates)
    
    # 确保High >= max(Open, Close), Low <= min(Open, Close)
    stock_data['High'] = stock_data[['Open', 'Close', 'High']].max(axis=1)
    stock_data['Low'] = stock_data[['Open', 'Close', 'Low']].min(axis=1)
    
    print("使用模拟数据演示")


In [ ]:
# K线形态识别函数
def identify_candle_patterns(data, lookback=3):
    """识别K线形态"""
    patterns = []
    
    for i in range(lookback, len(data)):
        # 获取最近几根K线
        recent = data.iloc[i-lookback+1:i+1]
        
        if len(recent) < lookback:
            patterns.append('')
            continue
        
        # 单根K线形态识别
        if i == len(data) - 1:  # 只识别最后一根
            row = recent.iloc[-1]
            open_price = row['Open']
            close_price = row['Close']
            high_price = row['High']
            low_price = row['Low']
            
            body = abs(close_price - open_price)
            upper_shadow = high_price - max(open_price, close_price)
            lower_shadow = min(open_price, close_price) - low_price
            total_range = high_price - low_price
            
            if body < total_range * 0.1:
                if upper_shadow > lower_shadow * 2:
                    patterns.append('倒T字线')
                elif lower_shadow > upper_shadow * 2:
                    patterns.append('T字线')
                else:
                    patterns.append('十字星')
            elif close_price > open_price:
                if body > total_range * 0.7:
                    patterns.append('大阳线')
                elif lower_shadow > body * 2:
                    patterns.append('锤子线')
                else:
                    patterns.append('普通阳线')
            else:
                if body > total_range * 0.7:
                    patterns.append('大阴线')
                elif upper_shadow > body * 2:
                    patterns.append('倒锤子线')
                else:
                    patterns.append('普通阴线')
        else:
            patterns.append('')
    
    # 三根K线组合识别（早晨之星、黄昏之星）
    for i in range(2, len(data)):
        if i < len(patterns):
            k1 = data.iloc[i-2]
            k2 = data.iloc[i-1]
            k3 = data.iloc[i]
            
            # 早晨之星
            if (k1['Close'] < k1['Open'] and  # 第一根阴线
                abs(k2['Close'] - k2['Open']) < (k2['High'] - k2['Low']) * 0.3 and  # 第二根小实体
                k3['Close'] > k3['Open'] and  # 第三根阳线
                k3['Close'] > (k1['Open'] + k1['Close']) / 2):  # 收盘价超过第一根中点
                patterns[i] = '早晨之星 ⭐'
            
            # 黄昏之星
            elif (k1['Close'] > k1['Open'] and  # 第一根阳线
                  abs(k2['Close'] - k2['Open']) < (k2['High'] - k2['Low']) * 0.3 and  # 第二根小实体
                  k3['Close'] < k3['Open'] and  # 第三根阴线
                  k3['Close'] < (k1['Open'] + k1['Close']) / 2):  # 收盘价低于第一根中点
                patterns[i] = '黄昏之星 ⭐'
    
    return patterns

# 识别K线形态
recent_data = stock_data.tail(30)
patterns = identify_candle_patterns(recent_data)

# 创建结果DataFrame
pattern_df = pd.DataFrame({
    '日期': recent_data.index,
    '开盘': recent_data['Open'].round(2),
    '收盘': recent_data['Close'].round(2),
    '最高': recent_data['High'].round(2),
    '最低': recent_data['Low'].round(2),
    'K线形态': patterns
})

# 显示识别到的形态
identified = pattern_df[pattern_df['K线形态'] != '']
if len(identified) > 0:
    print("识别到的K线形态：")
    print(identified[['日期', '开盘', '收盘', 'K线形态']].to_string(index=False))
else:
    print("未识别到明显的K线形态（可以尝试其他股票或时间段）")

# 绘制K线图并标注形态
fig, ax = plt.subplots(figsize=(15, 8))
mpf.plot(recent_data, type='candle', style='charles', ax=ax, volume=False)

# 标注识别到的形态
for idx, row in identified.iterrows():
    ax.annotate(row['K线形态'], 
                xy=(recent_data.index.get_loc(idx), row['最高']),
                xytext=(recent_data.index.get_loc(idx), row['最高'] * 1.05),
                ha='center', fontsize=9, fontweight='bold',
                arrowprops=dict(arrowstyle='->', color='red', lw=1.5))

ax.set_title('K线形态识别 - 实际股票数据', fontsize=16, fontweight='bold')
ax.set_xlabel('日期')
ax.set_ylabel('价格')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 💡 关键要点总结

1. **单根K线形态**：大阳线、大阴线、十字星、T字线等，反映单日多空力量对比
2. **双根K线组合**：吞没形态、刺透形态、乌云盖顶等，具有更强的信号意义
3. **三根K线组合**：早晨之星、黄昏之星、红三兵、三只乌鸦等，是最重要的反转和延续形态
4. **形态识别要点**：
   - 结合位置：形态在趋势中的位置很重要
   - 结合成交量：成交量配合可以增强信号可靠性
   - 结合趋势：顺势形态更可靠
5. **注意事项**：
   - 形态不是100%准确，需要结合其他分析方法
   - 在关键位置出现的形态更有意义
   - 多个形态相互确认可以提高准确性

## 🛠️ 工具与软件

### Python绘图工具

本Notebook使用的工具：
- **mplfinance**：专业K线图绘制库
- **matplotlib**：基础绘图库
- **pandas**：数据处理库
- **AKShare**：数据获取库

### 数据获取

- **AKShare**：支持A股、港股、美股数据，完全免费，无需梯子
- **Tushare**：A股数据，需要注册，免费版有数据限制
- **yfinance**：全球股票数据，可能需要梯子

### 注意事项

- ✅ **代码可运行**：所有代码都可以直接运行
- ✅ **数据获取**：如果AKShare获取失败，代码会自动使用模拟数据
- ✅ **网络要求**：AKShare通常不需要梯子，yfinance可能需要
- ⚠️ **形态识别**：自动识别可能不完美，需要人工验证

## 🔗 相关知识点

- [K线图基本构成](./K线图基本构成.ipynb)
- [趋势指标](../02_技术指标/趋势指标.md)
- [图表形态](../03_图表形态/README.md)

## 📚 拓展阅读

- 《日本蜡烛图技术》- K线图经典教材
- 《股票技术分析》- 深入学习技术分析
- 各交易软件帮助文档 - 了解软件使用方法
